# Customizing the Legend

In [1]:
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/mpg.csv")
print(df.shape)
df.head()

(234, 12)


,Unnamed: 0,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
0,1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
1,2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
2,3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
3,4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
4,5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


## Scale Parameters Related to the Legend

In [4]:
# Default legend
p1 = ggplot(df, aes("cty", "hwy")) + geom_point(aes(color="cyl"))
p1

In [5]:
# Legend name
p1 + scale_color_continuous(name="Number of cylinders")

In [6]:
# Breaks and labels
p1 + scale_color_continuous(breaks=[4, 5, 6, 8], labels=["four", "five", "six", "eight"])

In [7]:
# Format labels
p1 + scale_color_continuous(format="{} cylinders\nin the vehicle’s engine")

## Guides

### `guide_legend()`

In [8]:
# Default categorical legend
p2 = ggplot(df, aes("displ", "hwy", color="manufacturer")) + geom_point(size=5)
p2

In [9]:
# Legend name
p2 + scale_color_discrete(guide=guide_legend("Company"))

In [10]:
# Layout the legend in two columns
p2 + scale_color_discrete(guide=guide_legend(ncol=2))

In [11]:
# Fill by rows
p2 + scale_color_discrete(guide=guide_legend(ncol=2, byrow=True))

### `guide_colorbar()`

In [12]:
# Default color legend
p3 = ggplot(df, aes("displ", "hwy")) + geom_point(aes(color="cty"))
p3

In [13]:
# Legend name
p3 + scale_color_continuous(guide=guide_colorbar("City mileage"))

In [14]:
# Adjust colorbar size
p3 + scale_color_continuous(guide=guide_colorbar(barwidth=10, barheight=200))

In [15]:
# Fewer bins
p3 + scale_color_continuous(breaks=[13, 22, 31], guide=guide_colorbar(nbin=3))

### `guides()`

In [16]:
# Default complex legend
p4 = ggplot(df, aes("displ", "hwy")) + geom_point(aes(color="cty", shape="drv"), size=5)
p4

In [17]:
# Guides for 'color' and 'shape' aesthetics
p4 + guides(color=guide_colorbar(barwidth=10), shape=guide_legend(ncol=2))

### Customizing Aesthetics Appearance with `override_aes`

In [18]:
p4 + guides(shape=guide_legend(override_aes={'size': 8, 'color': "#f03b20"}))